In [ ]:
#this will have extended implementation of basic Policy gradient function and here we will have additional
#network function neural network as a critic to judge the actor's actions and improve its odds of balancing the CartPole
#this is insipred from open AI article on PPO and below research paper:
https://arxiv.org/pdf/1707.06347


In [ ]:
#only thing that is special in PPo is that critic sends an advantage function to the actor and then
based on the ratio between new policy and old policy ( this is the advantage fxn) we clip the update to a certan range and then run the update
this helps in avoiding too much drastic change in network and kind of adds a slow learning ensuring it learns better in time 

In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Categorical

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
lmbda         = 0.95
eps_clip      = 0.1
K_epoch       = 3
T_horizon     = 20 # Number of steps to collect before training

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


class PPO(nn.Module):
    def __init__(self):
        super(PPO, self).__init__()
        self.data = []

        self.fc1   = nn.Linear(4,256)
        self.fc_pi = nn.Linear(256,2)
        self.fc_v  = nn.Linear(256,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self, x, softmax_dim = 0):
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim)
        return prob

    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v

    def put_data(self, transition):
        self.data.append(transition)

    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, done_lst = [], [], [], [], [], []
        for transition in self.data:
            s, a, r, s_prime, prob_a, done = transition

            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            prob_a_lst.append([prob_a])
            done_mask = 0 if done else 1
            done_lst.append([done_mask])

        s,a,r,s_prime,done_mask, prob_a = torch.tensor(s_lst, dtype=torch.float).to(device), torch.tensor(a_lst).to(device), \
                                          torch.tensor(r_lst).to(device), torch.tensor(s_prime_lst, dtype=torch.float).to(device), \
                                          torch.tensor(done_lst, dtype=torch.float).to(device), torch.tensor(prob_a_lst).to(device)
        self.data = []
        return s, a, r, s_prime, done_mask, prob_a

    def train_net(self):
        s, a, r, s_prime, done_mask, prob_a = self.make_batch()

        for i in range(K_epoch):
            td_target = r + gamma * self.v(s_prime) * done_mask
            delta = td_target - self.v(s)
            delta = delta.detach().cpu().numpy() # Move delta to CPU for numpy conversion

            advantage_lst = []
            advantage = 0.0
            for delta_t in delta[::-1]:
                advantage = gamma * lmbda * advantage + delta_t[0]
                advantage_lst.append([advantage])
            advantage_lst.reverse()
            advantage = torch.tensor(advantage_lst, dtype=torch.float).to(device) # Move advantage back to GPU

            pi = self.pi(s, softmax_dim=1)
            pi_a = pi.gather(1,a)
            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == exp(log(a)-log(b))

            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(self.v(s) , td_target.detach())

            self.optimizer.zero_grad()
            loss.mean().backward()
            self.optimizer.step()

def main():
    env = gym.make('CartPole-v1')
    model = PPO().to(device) # Move model to device
    score = 0.0
    print_interval = 20

    for n_epi in range(2500):
        s, _ = env.reset()
        done = False
        while not done:
            for t in range(T_horizon): #20 max as 500 is taking some time
                prob = model.pi(torch.from_numpy(s).float().to(device)) # Move input to device
                m = Categorical(prob.cpu()) # Moved probabilities to CPU for Categorical sampling, 
                a = m.sample().item()
                s_prime, r, done, truncated, info = env.step(a)

                model.put_data((s, a, r/100.0, s_prime, prob[a].item(), done)) # Data is collected as numpy/python
                s = s_prime

                score += r
                if done:
                    break

            model.train_net()

        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0

    env.close()

    # Save the model's state dictionary
    torch.save(model.state_dict(), 'ppo_cartpole_policy.pth')
    print("PPO Model saved to ppo_cartpole_policy.pth")


if __name__ == '__main__':
    main()

In [ ]:
import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
from gymnasium.wrappers import RecordVideo
import base64
import io
from IPython.display import HTML
import glob
import os

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


# Instantiate the PPO model
model = PPO().to(device)

# Load the saved model state dictionary
try:
    model.load_state_dict(torch.load('ppo_cartpole_policy.pth', map_location=device))
    print("PPO Model loaded successfully.")
except FileNotFoundError:
    print("Error: PPO model file 'ppo_cartpole_policy.pth' not found. Please run the PPO training cell first to save the model.")
except Exception as e:
    print(f"An error occurred while loading the PPO model: {e}")

# Created the environment with the RecordVideo wrapper
# The video will be saved in a folder named 'videos'
env_video = RecordVideo(gym.make('CartPole-v1', render_mode='rgb_array'), video_folder='ppo_videos', episode_trigger=lambda x: x % 1 == 0) # Record every episode


num_episodes_to_record = 5 
for i in range(num_episodes_to_record):
    s, info = env_video.reset()
    done = False
    truncated = False
    while not done and not truncated:
        
        model.eval()
        with torch.no_grad():
            prob = model.pi(torch.from_numpy(s).float().to(device)) # Move input to device
        m = Categorical(prob.cpu()) # Move probabilities to CPU for Categorical sampling
        a = m.sample().item()
        s_prime, r, done, truncated, info = env_video.step(a)
        s = s_prime

env_video.close()

# Function to display the video in the notebook
def display_video(video_path):
    mp4 = open(video_path, 'rb').read()
    data_url = "data:video/mp4;base64," + base64.b64encode(mp4).decode()
    return HTML("""
    <video width="640" height="480" controls>
        <source src="%s" type="video/mp4">
    </video>
    """ % data_url)

# Display the last recorded video
# The RecordVideo wrapper saves videos with a specific naming convention (e.g., 'rl-video-episode-0.mp4')
# We can find the latest video file in the specified video folder
video_folder = 'ppo_videos'
list_of_files = glob.glob(os.path.join(video_folder, '*.mp4'))
latest_file = max(list_of_files, key=os.path.getctime) if list_of_files else None

if latest_file:
    print(f"Displaying video: {latest_file}")
    display_video(latest_file)
else:
    print("No video found in the 'ppo_videos' folder.")

In [ ]:
#This cell compares the performance of Reinforce Algorithm V/s PPO
#Above trained models weights are loaded in the colab notebook and run.
"""
Observations:
even though PPo waas trained for 250 episodes it showed more maturity and gave better Results
this will likely outperform more if trained with 10K episodes.
TODO - THink about Standard deviation and see how we can reduce it in PPO 
"""
"""
Sample Results
--- Evaluation Results ---

Algorithm: REINFORCE
--------------------
Number of Episodes: 100
Average Reward: 21.83
Reward Standard Deviation: 11.41
Maximum Reward: 71.0
Minimum Reward: 9.0
Success Rate (>= 475): 0.00%
Average Episode Length: 21.83

Algorithm: PPO
------------
Number of Episodes: 100
Average Reward: 194.49
Reward Standard Deviation: 30.22
Maximum Reward: 298.0
Minimum Reward: 140.0
Success Rate (>= 475): 0.00%
Average Episode Length: 194.49

--- Comparison Summary ---
PPO achieved a higher average reward than REINFORCE.
REINFORCE exhibited more consistent performance (lower standard deviation).

"""

import gymnasium as gym
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions import Categorical
import numpy as np

#these classes need to be defined if they are running in diff notebook
class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.softmax(self.fc2(x), dim=0)
        return x


class PPO(nn.Module):
    def __init__(self):
        super(PPO, self).__init__()
        self.fc1   = nn.Linear(4,256)
        self.fc_pi = nn.Linear(256,2)
        self.fc_v  = nn.Linear(256,1)

    def pi(self, x, softmax_dim = 0):
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim=softmax_dim)
        return prob

    def v(self, x):
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        return v

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load Trained Models
reinforce_policy = Policy().to(device)
ppo_model = PPO().to(device)

try:
    reinforce_policy.load_state_dict(torch.load('reinforce_cartpole_policy.pth', map_location=device))
    print("REINFORCE model loaded successfully.")
except FileNotFoundError:
    print("Error: REINFORCE model file 'reinforce_cartpole_policy.pth' not found.")
except Exception as e:
    print(f"An error occurred while loading the REINFORCE model: {e}")

try:
    ppo_model.load_state_dict(torch.load('ppo_cartpole_policy.pth', map_location=device))
    print("PPO model loaded successfully.")
except FileNotFoundError:
    print("Error: PPO model file 'ppo_cartpole_policy.pth' not found.")
except Exception as e:
    print(f"An error occurred while loading the PPO model: {e}")

reinforce_policy.eval()
ppo_model.eval()

env = gym.make('CartPole-v1')


# Run Evaluation Episodes and Collect Metrics
num_eval_episodes = 100
reinforce_rewards = []
ppo_rewards = []

print("\nEvaluating REINFORCE Policy...")
with torch.no_grad():
    for _ in range(num_eval_episodes):
        s, _ = env.reset()
        done = False
        truncated = False
        episode_reward = 0
        while not done and not truncated:
            prob = reinforce_policy(torch.from_numpy(s).float().to(device))
            m = Categorical(prob)
            a = m.sample().item()
            s_prime, r, done, truncated, info = env.step(a)
            s = s_prime
            episode_reward += r
        reinforce_rewards.append(episode_reward)

print("Evaluating PPO Policy...")
with torch.no_grad():
    for _ in range(num_eval_episodes):
        s, _ = env.reset()
        done = False
        truncated = False
        episode_reward = 0
        while not done and not truncated:
            prob = ppo_model.pi(torch.from_numpy(s).float().to(device))
            m = Categorical(prob.cpu()) # Sample on CPU as Categorical is CPU-based
            a = m.sample().item()
            s_prime, r, done, truncated, info = env.step(a)
            s = s_prime
            episode_reward += r
        ppo_rewards.append(episode_reward)

env.close()

# Calculate Metrics
reinforce_avg_reward = np.mean(reinforce_rewards)
reinforce_std_reward = np.std(reinforce_rewards)
reinforce_max_reward = np.max(reinforce_rewards)
reinforce_min_reward = np.min(reinforce_rewards)
reinforce_success_rate = np.sum(np.array(reinforce_rewards) >= 475) / num_eval_episodes * 100
reinforce_avg_episode_length = reinforce_avg_reward # For CartPole, reward is 1 per step

ppo_avg_reward = np.mean(ppo_rewards)
ppo_std_reward = np.std(ppo_rewards)
ppo_max_reward = np.max(ppo_rewards)
ppo_min_reward = np.min(ppo_rewards)
ppo_success_rate = np.sum(np.array(ppo_rewards) >= 475) / num_eval_episodes * 100
ppo_avg_episode_length = ppo_avg_reward # For CartPole, reward is 1 per step



print("\n--- Evaluation Results ---")

print("\nAlgorithm: REINFORCE")
print("--------------------")
print(f"Number of Episodes: {num_eval_episodes}")
print(f"Average Reward: {reinforce_avg_reward:.2f}")
print(f"Reward Standard Deviation: {reinforce_std_reward:.2f}")
print(f"Maximum Reward: {reinforce_max_reward}")
print(f"Minimum Reward: {reinforce_min_reward}")
print(f"Success Rate (>= 475): {reinforce_success_rate:.2f}%")
print(f"Average Episode Length: {reinforce_avg_episode_length:.2f}")


print("\nAlgorithm: PPO")
print("------------")
print(f"Number of Episodes: {num_eval_episodes}")
print(f"Average Reward: {ppo_avg_reward:.2f}")
print(f"Reward Standard Deviation: {ppo_std_reward:.2f}")
print(f"Maximum Reward: {ppo_max_reward}")
print(f"Minimum Reward: {ppo_min_reward}")
print(f"Success Rate (>= 475): {ppo_success_rate:.2f}%")
print(f"Average Episode Length: {ppo_avg_episode_length:.2f}")

print("\n--- Comparison Summary ---")
if ppo_avg_reward > reinforce_avg_reward:
    print("PPO achieved a higher average reward than REINFORCE.")
elif reinforce_avg_reward > ppo_avg_reward:
    print("REINFORCE achieved a higher average reward than PPO.")
else:
    print("Both algorithms achieved similar average rewards.")

#check for below conditoins for diff hyperparamters (#TODO)
#if ppo_std_reward < reinforce_std_reward:
  

#if ppo_success_rate > reinforce_success_rate:
#    print("PPO was more successful in reaching the 'solved' criteria.")
#elif reinforce_success_rate > ppo_success_rate:
#    print("REINFORCE was more successful in reaching the 'solved' criteria.")